In [ ]:
# Librerías y configuración
library(tidyverse)
library(data.table)
library(ggplot2)

# Tema y paleta global
theme_set(theme_minimal() + theme(
  plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
  plot.subtitle = element_text(size = 11, hjust = 0.5, color = "gray40"),
  axis.title = element_text(size = 11, face = "bold"),
  panel.grid.major.x = element_blank(),
  panel.grid.minor = element_blank()
))

out_dir <- "../output"

## a) Carga de datos y preparación

In [ ]:
# Descargar datos desde URL
url <- "https://raw.githubusercontent.com/LOST-STATS/LOST-STATS.github.io/master/Model_Estimation/Data/Event_Study_DiD/bacon_example.csv"
df <- fread(url)

# Definir año de tratamiento (group) y bandera tratado
# _nfd = 0 son nunca tratados
# g = año de primera adopción para unidades tratadas

df <- df %>%
  mutate(
    group = as.integer(_nfd),
    treated = if_else(group > 0, 1L, 0L)
  )

cat("Forma del dataset:", nrow(df), "x", ncol(df), "\n")
cat("Años de tratamiento (group):", paste(sort(unique(df$group)), collapse = ", "), "\n")
cat("Rango temporal:", min(df$year), "-", max(df$year), "\n")

head(df)

## b) Función para calcular ATT(g,t) y agregaciones

In [ ]:
# Función para calcular ATT(g,t) usando metodología Callaway & Sant'Anna
compute_att_gt <- function(data, outcome = "asmrs") {
  records <- list()
  idx <- 1
  
  treated_groups <- sort(setdiff(unique(data$group), 0))
  years <- sort(unique(data$year))
  
  for (g in treated_groups) {
    # Años donde ya está tratado ese grupo (t >= g)
    possible_t <- years[years >= g]
    
    for (t in possible_t) {
      # Año pre base: t0 = g - 1
      t0 <- g - 1
      
      # Datos del grupo tratado en t y t0
      treated_t <- data %>% filter(group == g, year == t)
      treated_t0 <- data %>% filter(group == g, year == t0)
      
      if (nrow(treated_t) == 0 || nrow(treated_t0) == 0) next
      
      # Grupos de control: nunca tratados (group = 0) o aún no tratados (group > t)
      control_t <- data %>% filter((group == 0 | group > t), year == t)
      control_t0 <- data %>% filter((group == 0 | group > t), year == t0)
      
      if (nrow(control_t) == 0 || nrow(control_t0) == 0) next
      
      # Diferencia en diferencias
      treated_delta <- mean(treated_t[[outcome]]) - mean(treated_t0[[outcome]])
      control_delta <- mean(control_t[[outcome]]) - mean(control_t0[[outcome]])
      att <- treated_delta - control_delta
      
      records[[idx]] <- data.frame(
        group = g,
        year = t,
        event_time = t - g,
        att_gt = att,
        n_treated = nrow(treated_t),
        n_control = nrow(control_t)
      )
      idx <- idx + 1
    }
  }
  
  return(bind_rows(records))
}

# Calcular ATT(g,t)
att_gt <- compute_att_gt(df)
cat("ATT(g,t) calculados:", nrow(att_gt), "observaciones\n\n")
print(head(att_gt, 10))

### Agregaciones por grupo, período y tiempo relativo

In [ ]:
# Definir rutas de salida
tt_out <- file.path(out_dir, "salida_preg2_1.csv")
tg_out <- file.path(out_dir, "salida_preg2_2.csv")
tp_out <- file.path(out_dir, "salida_preg2_3.csv")
te_out <- file.path(out_dir, "salida_preg2_4.csv")

# Agregaciones
att_by_group <- att_gt %>%
  group_by(group) %>%
  summarise(att_group = mean(att_gt), .groups = "drop")

att_by_period <- att_gt %>%
  group_by(year) %>%
  summarise(att_period = mean(att_gt), .groups = "drop")

att_by_event <- att_gt %>%
  group_by(event_time) %>%
  summarise(att_event = mean(att_gt), .groups = "drop") %>%
  arrange(event_time)

# Guardar tablas
write.csv(att_gt, tt_out, row.names = FALSE)
write.csv(att_by_group, tg_out, row.names = FALSE)
write.csv(att_by_period, tp_out, row.names = FALSE)
write.csv(att_by_event, te_out, row.names = FALSE)

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("TABLAS DE AGREGACIONES GUARDADAS\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
cat("Archivo 1 (ATT por grupo/período/tiempo):", tt_out, "\n")
cat("Archivo 2 (ATT agregado por grupo):", tg_out, "\n")
cat("Archivo 3 (ATT agregado por período):", tp_out, "\n")
cat("Archivo 4 (ATT agregado por tiempo relativo):", te_out, "\n")
cat(paste(rep("=", 70), collapse = ""), "\n\n")

cat("\n--- ATT agregado por tiempo relativo (comparable con Event-Study TWFE) ---\n")
print(att_by_event)

### Interpretación de agregaciones

- **Por grupo (ATT(g,·))**: promedio de efectos para cada cohorte g (año de adopción). Mide heterogeneidad entre cohortes.
- **Por período (ATT(·,t))**: promedio entre cohortes activas en año t. Captura shocks temporales del tratamiento.
- **Por tiempo relativo (ATT(e))**: promedio por distancia al tratamiento (event_time = t−g). Es lo más comparable con los coeficientes del Event-Study TWFE.

## c) Comparación CSDiD vs Event-Study TWFE

In [ ]:
# Leer resultado del Event-Study TWFE de Pregunta 1
es_path <- file.path(out_dir, "salida_2.csv")

if (file.exists(es_path)) {
  es_twfe <- read.csv(es_path) %>%
    rename(
      twfe_coef = estimate,
      twfe_se = se,
      twfe_ci_lower = ci_lower,
      twfe_ci_upper = ci_upper
    ) %>%
    select(event_time, twfe_coef, twfe_se, twfe_ci_lower, twfe_ci_upper)
  
  # Fusionar CSDiD con TWFE
  comparison <- att_by_event %>%
    full_join(es_twfe, by = "event_time") %>%
    arrange(event_time)
  
  comp_out <- file.path(out_dir, "salida_preg2_5.csv")
  write.csv(comparison, comp_out, row.names = FALSE)
  
  cat(paste(rep("=", 85), collapse = ""), "\n")
  cat("COMPARACIÓN CSDiD vs EVENT-STUDY TWFE\n")
  cat(paste(rep("=", 85), collapse = ""), "\n")
  print(comparison)
  cat(paste(rep("=", 85), collapse = ""), "\n")
  cat("\n✓ Tabla guardada en:", comp_out, "\n")
  
} else {
  cat("ADVERTENCIA: No se encontró", es_path, "\n")
  cat("Omitiendo comparación con Event-Study TWFE.\n")
  es_twfe <- data.frame()
  comparison <- data.frame()
}

## d) Visualización comparada

In [ ]:
# Gráfico combinado comparando CSDiD vs TWFE
if (nrow(es_twfe) > 0) {
  
  p <- ggplot() +
    # Línea horizontal en cero
    geom_hline(yintercept = 0, linetype = "dashed", color = "gray50", linewidth = 0.5) +
    # TWFE con bandas de error (intervalos de confianza)
    geom_ribbon(
      data = es_twfe,
      aes(x = event_time, ymin = twfe_ci_lower, ymax = twfe_ci_upper),
      fill = "darkorange",
      alpha = 0.2,
      color = NA
    ) +
    geom_point(
      data = es_twfe,
      aes(x = event_time, y = twfe_coef, color = "TWFE"),
      size = 3,
      shape = 1
    ) +
    geom_line(
      data = es_twfe,
      aes(x = event_time, y = twfe_coef, color = "TWFE"),
      linewidth = 0.6,
      alpha = 0.7
    ) +
    # CSDiD ATT por event_time
    geom_point(
      data = att_by_event,
      aes(x = event_time, y = att_event, color = "CSDiD"),
      size = 3
    ) +
    geom_line(
      data = att_by_event,
      aes(x = event_time, y = att_event, color = "CSDiD"),
      linewidth = 0.6,
      alpha = 0.7
    ) +
    # Estética
    scale_color_manual(
      values = c("TWFE" = "darkorange", "CSDiD" = "navy"),
      name = "Método"
    ) +
    labs(
      title = "Comparación: CSDiD vs Event-Study TWFE",
      subtitle = "Efectos dinámicos por tiempo relativo al tratamiento",
      x = "Tiempo relativo al tratamiento (años)",
      y = "Efecto sobre ASMRS",
      caption = "CSDiD: Callaway & Sant'Anna | TWFE: Two-Way Fixed Effects (con ±95% IC)"
    ) +
    theme(
      legend.position = "top",
      panel.grid.major.x = element_blank()
    )
  
  print(p)
  
  plot_out <- file.path(out_dir, "salida_preg2_6.png")
  ggsave(plot_out, plot = p, width = 12, height = 7, dpi = 300)
  cat("\n✓ Gráfico de comparación guardado en:", plot_out, "\n")
  
} else {
  cat("No se generó gráfico combinado porque no se encontró salida_2.csv\n")
}

## Resumen y discusión

### Características de CSDiD:
- **Control válido**: Usa como grupo de control los no tratados o aún no tratados en el período t → evita comparaciones prohibidas
- **ATT(g,t)**: Efecto específico del grupo de adopción g en el período t
- **Agregaciones**: Permite descomponer la heterogeneidad del tratamiento por cohorte, período o tiempo relativo

### Comparación con TWFE Event-Study:
- TWFE puede incluir comparaciones prohibidas (tratado vs tratado en otro período) → pesos negativos
- CSDiD evita esto siendo explícito sobre el grupo de control en cada momento
- **Agregación por tiempo relativo**: La más comparable entre ambos métodos
- Diferencias entre CSDiD y TWFE sugieren presencia de heterogeneidad del tratamiento

### Archivos generados:
1. `salida_preg2_1.csv`: Matriz ATT(g,t) completa
2. `salida_preg2_2.csv`: Agregación por grupo de adopción
3. `salida_preg2_3.csv`: Agregación por período
4. `salida_preg2_4.csv`: Agregación por tiempo relativo
5. `salida_preg2_5.csv`: Tabla de comparación CSDiD vs TWFE
6. `salida_preg2_6.png`: Gráfico comparativo